# Causation Corpus classification 
#### Chain of Thoughts Self Consistency

This notebook uses the chain-of-thoughts self consistency prompting technique to classify sentences via LLMs.

## OpenAI Privacy Policy
This notebook uses OpenAI's API, meaning that your data will be sent to the OpenAI servers.

For concerns about how your data will be handled, please read through the Privacy Policy [here](https://openai.com/policies/api-data-usage-policies).

## Notebook Overview

In [ ]:
# openai api key

In [ ]:
# model set up
# sampling scheme

In [ ]:
# prompt set up
# upload the prompt file
# validate: validate prompt file.
# feedback: collapsible/viewable widget for what's uploaded.

## Classification

In [ ]:
# upload the dataset
# feedback: summary of what's been uploaded. Which column is used for classification.

In [ ]:
# user input: confirmation
# + how many classifications will be run.
# + tikdollar set up
# + expected columns in the output.xlsx.
# + pop up view of the prompt + dataset summary + model config + tikdollar + expected output.xlsx
# + feedback: run -> green -> running -> progress bar.
# run classification
# expensive compute: progressive checkpointing - 
# feedback: completion of classification -> how much is spent -> download xlsx.